In [1]:
# Packages for data analysis
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('data/100tweets.csv', sep=';')
print(df.shape)
df.head()

(100, 18)


,Column1,...1,Row.names,rowid,lat2,lon2,user_id,Column2,text2,retweeted,lang,id,created_at,noChar,iso_lang_1,prob_1,text1,classname
0,4,53090,93493,319774,49.302954,7.466072,434326976,NaN,"Not good, not bad...disappointing",False,en,"1,19E+18",29.10.2019 16:55,34,en,0.881157,"Not good, not bad...disappointing\r\n#MartinaP...",home
1,6,4340,117041,460051,49.641076,6.06006735,603021612,NaN,thats so poopoo,False,en,"3,30E+17",03.05.2013 16:03,15,en,0.862470,thats so poopoo,other
2,8,9104,145546,679738,49.611273,6.12770012,1298568337,NaN,"Monday, June , : Spaghetti Bolognese",False,it,"3,42E+17",03.06.2013 11:22,37,en,0.914869,"Monday, June 3, 2013: Spaghetti Bolognese http...",home
3,24,23743,25333,105500,49.658195,6.14052043,28345806,NaN,Where are the people who left on the boat?? Do...,False,en,"5,63E+17",05.02.2015 21:53,70,en,0.964202,Where are the people who left on the boat?? Do...,home
4,38,55020,97806,347149,63.833300,20.25,456045381,NaN,Christmas cosiness. Merry very. |,False,sv,"8,13E+17",24.12.2016 21:21,34,en,0.924872,Christmas cosiness. Merry very. | #christmasev...,home


In [5]:
# Do not run if pre-processing is not required
!pip install tweet-preprocessor

In [6]:
# Do not run if pre-processing is not required
import preprocessor as prep

In [7]:
# Do not run if pre-processing is not required
## Set options for the tweet-preprocessor
prep.set_options(prep.OPT.URL, prep.OPT.RESERVED, prep.OPT.EMOJI, prep.OPT.SMILEY)

## Clean text and remove #,@ symbols
def clean_tweet(text):
    text = prep.clean(text)
    table = str.maketrans('','','#@')
    return text.translate(table)

In [8]:
# Do not run if pre-processing is not required
df['text_processed'] = df['text1'].apply(lambda x: clean_tweet(x))

In [9]:
# Do not run if pre-processing is not required
print(df.shape)
df.head()

(100, 19)


,Column1,...1,Row.names,rowid,lat2,lon2,user_id,Column2,text2,retweeted,lang,id,created_at,noChar,iso_lang_1,prob_1,text1,classname,text_processed
0,4,53090,93493,319774,49.302954,7.466072,434326976,NaN,"Not good, not bad...disappointing",False,en,"1,19E+18",29.10.2019 16:55,34,en,0.881157,"Not good, not bad...disappointing\r\n#MartinaP...",home,"Not good, not bad...disappointing MartinaPadbe..."
1,6,4340,117041,460051,49.641076,6.06006735,603021612,NaN,thats so poopoo,False,en,"3,30E+17",03.05.2013 16:03,15,en,0.862470,thats so poopoo,other,thats so poopoo
2,8,9104,145546,679738,49.611273,6.12770012,1298568337,NaN,"Monday, June , : Spaghetti Bolognese",False,it,"3,42E+17",03.06.2013 11:22,37,en,0.914869,"Monday, June 3, 2013: Spaghetti Bolognese http...",home,"Monday, June 3, 2013: Spaghetti Bolognese"
3,24,23743,25333,105500,49.658195,6.14052043,28345806,NaN,Where are the people who left on the boat?? Do...,False,en,"5,63E+17",05.02.2015 21:53,70,en,0.964202,Where are the people who left on the boat?? Do...,home,Where are the people who left on the boat?? Do...
4,38,55020,97806,347149,63.833300,20.25,456045381,NaN,Christmas cosiness. Merry very. |,False,sv,"8,13E+17",24.12.2016 21:21,34,en,0.924872,Christmas cosiness. Merry very. | #christmasev...,home,Christmas cosiness. Merry very. | christmaseve...


In [10]:
# transforming column with tweets to the list
sentences = list(df['text_processed'].values) # text will be the input to the model

In [11]:
# Do not forget to use GPU in Colab
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [12]:
from transformers import pipeline

In [13]:
# Applying classifier to the list of tweets
activity_classifier = pipeline("zero-shot-classification", model = 'facebook/bart-large-mnli')
activity_result = activity_classifier(list(sentences), candidate_labels = [
'Sleep waking up', 'Baby infant family', 'Reading books', 'TV YouTube Netflix', 'Dish recipy kitchen',

'Campus school university education study','Work job','Training secondment traineeship',
'Office lunch colleagues department director','Business industry research conferences and forums', 

'Outdoor sports recreation','Nature landscape wildlife','Walk hiking','Weather season rain cold',

'Delays passenger arrival departure','Cycling bikes scooters','Airport waiting lounge','Car petrol traffic',

'Entertainment','Religion','Groceries and shopping','Healthcare and well-being',
'Fitness club', 'Services brand retail','Café restaurants','City sightseeing','Art shows performances',

'Politics and affairs','Personality and relationships','Beliefs wishes kindness'],)
activity_result[10]

{'sequence': 'It was a real inspirational work?',
 'labels': ['Work job',
  'Entertainment',
  'Nature landscape wildlife',
  'City sightseeing',
  'Baby infant family',
  'Art shows performances',
  'Café restaurants',
  'Dish recipy kitchen',
  'Beliefs wishes kindness',
  'Campus school university education study',
  'Fitness club',
  'Personality and relationships',
  'Weather season rain cold',
  'Healthcare and well-being',
  'Business industry research conferences and forums',
  'Sleep waking up',
  'Office lunch colleagues department director',
  'Services brand retail',
  'Delays passenger arrival departure',
  'Outdoor sports recreation',
  'Training secondment traineeship',
  'Walk hiking',
  'Reading books',
  'Cycling bikes scooters',
  'Politics and affairs',
  'TV YouTube Netflix',
  'Car petrol traffic',
  'Airport waiting lounge',
  'Religion',
  'Groceries and shopping'],
 'scores': [0.2008398324251175,
  0.1515890210866928,
  0.04114386811852455,
  0.0358293317258358

In [14]:
np.argmax(activity_result[0]['scores']) 

0

In [15]:
# Leaving only first prediction for validation
y_activity_pred = [result['labels'][np.argmax(result['scores'])] for result in activity_result]
y_activity_pred[10]

'Work job'

In [16]:
# Replacing predictions with general categories for validation
for i in range(len(y_activity_pred)):
  
    if y_activity_pred[i] == 'Sleep waking up':
        y_activity_pred[i] = 'home'    
    if y_activity_pred[i] == 'Baby infant family':
        y_activity_pred[i] = 'home' 
    if y_activity_pred[i] == 'Reading books':
        y_activity_pred[i] = 'home' 
    if y_activity_pred[i] == 'TV YouTube Netflix':
        y_activity_pred[i] = 'home' 
    if y_activity_pred[i] == 'Dish recipy kitchen':
        y_activity_pred[i] = 'home' 

    if y_activity_pred[i] == 'Campus school university education study':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Work job':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Training secondment traineeship':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Office lunch colleagues department director':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Business industry research conferences and forums':
        y_activity_pred[i] = 'work'

    if y_activity_pred[i] == 'Outdoor sports recreation':
        y_activity_pred[i] = 'outdoors'
    if y_activity_pred[i] == 'Nature landscape wildlife':
        y_activity_pred[i] = 'outdoors'
    if y_activity_pred[i] == 'Weather season rain cold':
        y_activity_pred[i] = 'outdoors'
    if y_activity_pred[i] == 'Walk hiking':
        y_activity_pred[i] = 'outdoors'

    if y_activity_pred[i] == 'Delays passenger arrival departure':
        y_activity_pred[i] = 'transport'
    if y_activity_pred[i] == 'Cycling bikes scooters':
        y_activity_pred[i] = 'transport'
    if y_activity_pred[i] == 'Airport waiting lounge':
        y_activity_pred[i] = 'transport'
    if y_activity_pred[i] == 'Car petrol traffic':
        y_activity_pred[i] = 'transport'
        
    if y_activity_pred[i] == 'Entertainment':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Religion':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Groceries and shopping':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Healthcare and well-being':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Fitness club':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Services brand retail':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Café restaurants':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'City sightseeing':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Theatre shows performances':
        y_activity_pred[i] = 'services'

    if y_activity_pred[i] == 'Politics and affairs':
        y_activity_pred[i] = 'other'
    if y_activity_pred[i] == 'Personality and relationships':
        y_activity_pred[i] = 'other'
    if y_activity_pred[i] == 'Beliefs wishes kindness':
        y_activity_pred[i] = 'other'
  
# print list
print(y_activity_pred)

['home', 'outdoors', 'home', 'transport', 'other', 'other', 'other', 'services', 'other', 'outdoors', 'work', 'other', 'services', 'services', 'home', 'home', 'other', 'outdoors', 'services', 'other', 'services', 'other', 'outdoors', 'services', 'services', 'services', 'services', 'services', 'outdoors', 'outdoors', 'outdoors', 'services', 'services', 'services', 'services', 'services', 'services', 'home', 'services', 'services', 'transport', 'services', 'services', 'services', 'services', 'work', 'work', 'services', 'services', 'other', 'services', 'home', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'work', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'outdoors', 'services', 'transport', 'outdoors', 'services', 'transport', 'services', 'transport', 'transport', 'outdoors', 'transport', 'transport', 'transport', 'transport', 'work', 'services', 'services', 'work', 'work', 

In [17]:
y_activity_true = list(df['classname'].values) # true labels (annotated by human)

In [18]:
from sklearn.metrics import classification_report, confusion_matrix

In [19]:
# Classification report for the general categories (1 row left unannotated, will be fixed)
print(classification_report(y_activity_true, y_activity_pred))

              precision    recall  f1-score   support

        home       0.43      0.50      0.46         6
       other       0.55      0.38      0.44        16
    outdoors       0.36      0.40      0.38        10
    services       0.66      0.79      0.72        42
   transport       0.80      0.67      0.73        12
        work       0.64      0.50      0.56        14

    accuracy                           0.61       100
   macro avg       0.57      0.54      0.55       100
weighted avg       0.61      0.61      0.60       100



In [20]:
# Transforming list of predictions back to dataframe
df_result = pd.DataFrame(activity_result)

In [21]:
# Checking predictions
df_result.head()

,sequence,labels,scores
0,"Not good, not bad...disappointing MartinaPadbe...","[Reading books, Work job, Personality and rela...","[0.15626543760299683, 0.06222022324800491, 0.0..."
1,thats so poopoo,"[Weather season rain cold, Delays passenger ar...","[0.09300099313259125, 0.056102436035871506, 0...."
2,"Monday, June 3, 2013: Spaghetti Bolognese","[Dish recipy kitchen, Entertainment, Café rest...","[0.49419575929641724, 0.07745104283094406, 0.0..."
3,Where are the people who left on the boat?? Do...,"[Delays passenger arrival departure, Outdoor s...","[0.1846390813589096, 0.1719832718372345, 0.105..."
4,Christmas cosiness. Merry very. | christmaseve...,"[Beliefs wishes kindness, Personality and rela...","[0.22708377242088318, 0.16258980333805084, 0.0..."


In [22]:
# Saving prediction results to csv
df_result.to_csv('resultFB_cleaned.csv')